# ML căn bản part 3

## 4.5 OVERFITTING VÀ UNDERFITTING

**Under fitting** 

Khi ta vẫn có thể optimize giúp nâng độ chính xác của model trong bài toán generalize lên ( tuning model, preprocessing , feature processing, training ...)

Giải pháp :
- Giải pháp tốt nhất đối với under fitting : **More data**
- Sau đó : **tuning model**, nếu network chỉ có thể nhớ 1 số ít các thuộc tính, quá trình optimize sẽ **bắt** model tập trung vào những **thuộc tính đặc trưng nhất** --> tăng độ chính xác cho bài toán generalize

**Over fitting**

Ở các ví dụ trước, trong bước validate model, độ chính xác của model tăng đến 1 ngưỡng nhất định rồi sau 1 vài epoch bắt đầu giảm
--> Overfitting

2 yếu tố quan trọng để đánh giá 1 model :
- **Optimization** : Thiết kế và train Model để có được độ chính xác tối đa trong **tập training** ( Điểm số trên lớp cao nhất có thể )
- **Generalization** : Độ chính xác khi áp dụng model vào **data thực tế **( Lúc đi làm ) 

**Mục đích chính** của bài toán là **Generalization**, nhưng đây là yếu tố ta **không thể kiểm soát được**, ta chỉ "fit" được model với **training data**





### 4.5.1 Các giải pháp cho bài toán Over fitting

#### 1. GIẢM NETWORK SIZE

Cách đơn giản nhất là **giảm size** của model a.k.a giảm số **learnable parameters** của model ( model capacity ):
- Giảm số hidden layers
- Giảm số hidden units trong 1 layer

Hiểu đơn giản, model càng to não càng nhớ được nhiều, thậm chí nhớ cả cách mapping giữa train data và label, Tuy vậy độ chính xác khi gặp các data mới lại thấp --> Generalize thấp 

Nhưng nếu network không đủ to não để nhớ các thuộc tính quan trọng --> **Under fitting** 

Không có 1 công thức nào có thể chỉ ra đâu là số layer phù hợp, hay đâu là số hidden unit phù hợp cho mỗi layer --> Ta phải tự đánh giá (với tập validation) trên nhiều kiến trúc khác nhau để tìm ra size phù hợp --> Bắt đầu với ít layers và params, sau đó tăng dần size đến khi thấy độ chính xác không ngừng giảm  

#### 2. WEIGHT REGULARIZATION

Nguyên lý [Dao cạo Occam](https://vi.wikipedia.org/wiki/Dao_c%E1%BA%A1o_Ockham) : 
>"Lời giải thích đơn giản nhất thường là lời giải thích xác đáng nhất". - William xứ Occam

Điều này cũng đúng khi xây dựng network : có nhiều tập trọng số (weights) trong nhiều model khác nhau, nhưng những model đơn giản sẽ ít overfit hơn những model phức tạp   

Model đơn giản có thể hiểu như sau :
- Số các params ít ( như ta đã đề cập trước đó )
- Giá trị các params không chênh lệch quá lớn : *regularization*


Cách đơn giản để hiểu Regularize : ta để **1 trọng số cho độ phức tạp của network** và tìm cách để **minimize** giá trị đó bằng cách thêm vào loss function **1 cost** biểu thị độ phức tạp của network --> network càng phức tạp cost càng lớn

Cost này có 2 loại chính :
- L1 regularization : cost tính theo giá trị **tuyệt đối (abs)* vào các weights
- L2 regularization : cost thêm vào giá trị **bình phương (square)** của các weights ( **weight decay** )


Trong keras, weight regularization được thêm vào bằng cách cho thêm *weight regularizer instance* vào các layers :
```python
from keras import regularizers

model = model.Sequential()
model.add(layers.Dense(16, 
         kernel_regularizer=regularizers.l2(0.001),
         activation='relu', input_shape=(10000,)))
...
```

l2(0.001) có nghĩa là mỗi hệ số của layer sẽ thêm 0.001 * w vào tổng loss. Lưu ý rằng loss của network sẽ cao hơn nhiều lúc train so với lúc test

Regularize L2 trong Keras :
```python
from keras import regularizers
reg_model = models.Sequential()
reg_model.add(layers.Dense(16,
                           kernel_regularizer=regularizers.l2(0.001),
                          activation='relu',input_shape=(10000,)))
reg_model.add(layers.Dense(16,
                           kernel_regularizer=regularizers.l2(0.0001),
                          activation='relu'))
reg_model.add(layers.Dense(1,activation='sigmoid'))
```

![normal vs reg L2 model](regu.png)

Ta thấy model với L2 regularization ( dấu +) bị over fit ít hơn so với model thường ( chấm đỏ ), dù chúng có cùng số param

Ngoài L2 regularization, ta còn có thể sử dụng các regularizers sau của Keras:
```python
from keras import regularizers

regularizers.l1(0.001) # L1 regularization
regularizers.l1_l2(l1=0.001,l2=0.001) # L1 va L2 dong thoi
```

#### 3. ADDING DROPOUT

>**Dropout** is one	of the **most effective and most commonly used** *regularization technique* for neural networks, developed by Hinton and his students at the University of Toronto. [Wiki](https://en.wikipedia.org/wiki/Dropout_(neural_networks)

>[(Reddit ELI5)](https://www.reddit.com/r/MachineLearning/comments/4bfswx/eli5_dropout_technique/) On each epoch during training, turn off (set activations to 0) of randomly selected neurons with probability p, selected from Bernoulli distribution . By doing this:
- There are no 'free-rider' neurons, that do not do anything and rely on other neurons *giảm các neurons không đóng góp gì cho network*
- Reduces co-dependency between neurons -> reduces number of neurons that rely on each other for prediction. *Giảm các neuron phụ thuộc lẫn nhau*
- Increases training speed by stimulating that all neurons contribute individually *tăng tốc độ training* 
- Reduces chances of overfitting *giảm khả năng over-fitting*

**Drop rate** : % số feature bị cho = 0 trong quá trình training, thường trong khoảng 0.1-0.5. Khi test ta sẽ scale lại weight

E.g : giả sử layer_output là 1 ma trận 
- Quá trình train : tắt random 50% các giá trị trong ma trận
  ```python
# at training time, we drop out 50% of the units in the output
  layer_output *= np.randint(0,high=1,size=layer_output.shape)
  ```
- Quá trình test : scale lại các weight theo số đã bị drop ( khi mất đi 50% weights, các weight còn lại sẽ có giá trị lớn x2 lần )
  ```python
  # test time :
  layer_output *= 0.5
  ```




Lưu ý rằng ta cũng có thể cài đặt cả 2 phép toán trong quá trình train để khi test không cần scale lại :
```python
# at training time
layer_output *= np.randint(0, high=1, size= layer_output.shape)
layer_output /= 0.5 # scale up 
```

Trong keras, ta dùng Dropout như sau :
```python
model.add(layers.Dropout(0.5))
```

![Drop out vs normal model](drop.png) 


### Tổng kết lại :
Các cách để ngăn ngừa over-fitting :
- Thêm data
- Giảm size network
- Weight regularization ( L1, L2, L1_L2...)
- Drop out